# Revisiting the Strategies in Federated Learning

As mentioned in the previous unit, Strategies are at the core of federated learning. They determine how clients are selected, which updates are used, and how the new changes are aggregated.

In this unit, we will focus on custom Strategies. To begin, we need to set up the environment for this notebook's development.

### Exercise
As you are familiar with one of the deep learning frameworks, you can implement the following part based on your preference, either PyTorch, Tensorflow, or JAX.


In [1]:
import numpy as np
import tensorflow as tf
from typing import List, Dict, Optional, Tuple, Union


import flwr as fl
from flwr.common import Context
from flwr.client import ClientApp


#Load the CIFAR-10 in NUM_CLIENTS different subsets for the training and test as it has been in the previous unit

NUM_CLIENTS = 10

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s

# Code to load the dataset
def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into 'num_clients' partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])  
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    
    return train_ds, val_ds, test_ds


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

# Define the model to be used in the clients

# The part to adjust for each framework
def generate_ann():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    return model


def get_parameters(net) -> List[np.array]:
    return net.get_weights()


def set_parameters(net, parameters: List[np.ndarray]):
    net.set_weights(parameters)
    return net


def train(net, trainloader, epochs: int):
    net.fit(trainloader[0], trainloader[1], epochs=epochs, batch_size=32, steps_per_epoch=3)
    return net


def test(net, testloader):
    loss, accuracy = net.evaluate(testloader[0], testloader[1])
    return loss, accuracy

# Class to contain a Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        self.net = train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, Client {self.cid} accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(Context) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #get the identification
    partition_id = int(Context.node_config["partition-id"])
    #Take the appropiate part of the dataset
    trainloader = trainloaders[int(partition_id)]
    valloader = valloaders[int(partition_id)]
    #Create and return the Client
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()

/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/utils/file_utils.py:115: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  archive.extractall(


Considering the previous code, the model developed has several possibilities for implementing the Strategy object such as `FedAvg` or `FedAdagrad`,  as seen in the  previous Unit. For example, the following code should create a strategy. 

In [2]:
import flwr as fl
# Create an instance of the model and get the parameters
model = generate_ann()
params = get_parameters(model)

/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [3]:
from flwr.server import ServerApp, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.common import ndarrays_to_parameters

num_rounds = 3

def server_fn(context: Context):
    # Generate the model and parameters
    model = generate_ann()
    params = get_parameters(model)
    del model
    global_model_init = ndarrays_to_parameters(params)

    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,  
        fraction_evaluate=0.3,  
        min_fit_clients=3,
        min_evaluate_clients=2,
        min_available_clients=NUM_CLIENTS, 
        initial_parameters=global_model_init, # Initial parameters
    )

    # Define ServerConfig
    config = fl.server.ServerConfig(num_rounds=num_rounds)

    # Return the configuration and strategy for this server
    return ServerAppComponents(strategy=strategy, config=config)

# Create Server
server_app = ServerApp(server_fn=server_fn)

# Create Client
client_app = ClientApp(client_fn=client_fn)

#Start the simulation
fl.simulation.run_simulation(
    server_app=server_app, client_app=client_app, num_supernodes=NUM_CLIENTS
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=36264) [Client 7] fit, config: {}


(ClientAppActor pid=36264) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(ClientAppActor pid=36264)   super().__init__(**kwargs)
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1172 - loss: 2.359652
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0807 - loss: 2.4910  
(ClientAppActor pid=36264) [Client 0] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.0612 - loss: 2.4384
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0992 - loss: 2.456315
(ClientAppActor pid=36265) [Client 9] fit, config: {} [repeated 6x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0794 - loss: 2.4494   [repeated 7x across cluster]
(ClientAppActor pid=36262) [Client 2] evaluate, config: {} [repeated 5x across cluster]


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


1/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.1250 - loss: 2.3876
(ClientAppActor pid=36262) [Client 2] loss:2.4994595050811768, Client 2 accuracy:0.05999999865889549 [repeated 5x across cluster]
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0964 - loss: 2.3480


(raylet) [2025-02-16 17:34:41,435 E 36250 10716465] file_system_monitor.cc:116: /tmp/ray/session_2025-02-16_17-34-26_990380_36200 is over 95% full, available space: 22.9576 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.
(ClientAppActor pid=36262) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead. [repeated 2x across cluster]
(ClientAppActor pid=36262)   super().__init__(**kwargs) [repeated 2x across cluster]
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 15.46s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.324413776397705
INFO :      		round 2: 2.451725165049235
INFO :      		round 3: 2.3092896143595376
IN

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.0563 - loss: 2.4988
(ClientAppActor pid=36262) [Client 3] fit, config: {} [repeated 2x across cluster]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.1138 - loss: 2.2691 [repeated 5x across cluster]
(ClientAppActor pid=36262) [Client 5] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=36262) [Client 5] loss:2.2872626781463623, Client 5 accuracy:0.10999999940395355 [repeated 3x across cluster]


It may be worth mentioning that Flower, by default, initializes the global model by making a call to one random client before distributing it to the remaining clients. However, sometimes more control is required, such as when performing fine-tuning. In such situations, we use server-side initialization, and the `initial_parameters` parameter will hold the initial version of the model for all clients. It is important to note that this parameter must be a serialization of the data, so the utility function `ndarrays_to_parameters` can be quite handy in this case.

Now, let's move on to customizing the type of evaluation performed on the models. Broadly speaking, there are two possibilities: server-side evaluation and client-side evaluation.

**Centralized evaluation** (server-side) is similar to traditional machine learning, where the server holds a partition solely for evaluating the aggregated model. This approach reduces communication and is suitable for situations with limited bandwidth. There is no need to send the model to the clients for evaluation, and the entire evaluation dataset is available at all times.

**Federated evaluation** (client-side) is more complex, but it usually represents real-world scenarios more accurately. In this approach, the evaluation dataset is distributed among the clients, which means that we can leverage a larger dataset spread among the resources of the clients. However, this approach comes with a cost. Since we don't have a central dataset, we should be aware that our evaluation dataset can change over consecutive rounds of learning if some clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we don't change the model, we can see our evaluation results fluctuate over consecutive rounds. Additionally, this approach can significantly increase the number of communications because the models have to be distributed among the clients and retrieved for evaluation.

The previous code snippet is an example of Flower performing Federated evaluations, as it uses the `evaluation` function that is executed on each `Client` and later aggregated after being sent to the server. On the other hand, a Centralized evaluation could be performed with a similar approach, as shown in the following code snippet:


In [4]:
def get_test_loader(input_dataset):
    # The `evaluate` function will be by Flower called after every round
    def evaluate_fn(
        server_round: int, parameters: fl.common.NDArrays, 
        config: Dict[str, fl.common.Scalar]) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        # Load the test data
        dataset = input_dataset
        
        # Update the model with the latest parameters
        model = generate_ann()
        set_parameters(model, parameters)
    
        # Evaluate the model on the test dataset
        loss, accuracy = test(model, dataset)
    
        # Log the evaluation results
        print(f"Server-side evaluation round {server_round} with loss {loss} / accuracy {accuracy}")
        
        return loss, {"accuracy": accuracy}

    return evaluate_fn

def server_fn(context: Context):
    # Create FedAvg strategy
    strategy = fl.server.strategy.FedAvg(
            fraction_fit=0.3,  
            fraction_evaluate=0.3,  
            min_fit_clients=3,
            min_evaluate_clients=2,
            min_available_clients=NUM_CLIENTS, 
            initial_parameters=fl.common.ndarrays_to_parameters(params),
            evaluate_fn=get_test_loader(testloader[0]),  # Pass the evaluation function
    )

    # Define ServerConfig
    config = fl.server.ServerConfig(num_rounds=num_rounds)

    # Return the configuration and strategy for this server
    return ServerAppComponents(strategy=strategy, config=config)

# Create Server
server_app = ServerApp(server_fn=server_fn)

# Start the simulation
fl.simulation.run_simulation(
    server_app=server_app, client_app=client_app, num_supernodes=NUM_CLIENTS
)


INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0952 - loss: 2.3101


INFO :      initial parameters (loss, other metrics): 2.296900749206543, {'accuracy': 0.10000000149011612}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation round 0 with loss 2.296900749206543 / accuracy 0.10000000149011612


(ClientAppActor pid=36441) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(ClientAppActor pid=36441)   super().__init__(**kwargs)


...

INFO :      fit progress: (3, 2.300903081893921, {'accuracy': 0.05999999865889549}, 14.52031812502537)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation round 3 with loss 2.300903081893921 / accuracy 0.05999999865889549


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 15.96s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.380004644393921
INFO :      		round 2: 2.3416190147399902
INFO :      		round 3: 2.2980446815490723
INFO :      	History (loss, centralized):
INFO :      		round 0: 2.296900749206543
INFO :      		round 1: 2.328767776489258
INFO :      		round 2: 2.335635185241699
INFO :      		round 3: 2.300903081893921
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.10000000149011612),
INFO :      	              (1, 0.1899999976158142),
INFO :      	              (2, 0.10000000149011612),
INFO :      	              (3, 0.05999999865889549)]}
INFO :      


(ClientAppActor pid=36441) [Client 9] evaluate, config: {} [repeated 6x across cluster]
(ClientAppActor pid=36442) [Client 4] loss:2.282099485397339, Client 4 accuracy:0.15000000596046448 [repeated 5x across cluster]
(ClientAppActor pid=36442) [Client 5] fit, config: {} [repeated 2x across cluster]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.0921 - loss: 2.3491 [repeated 6x across cluster]
(ClientAppActor pid=36442) [Client 4] evaluate, config: {} [repeated 2x across cluster]
(ClientAppActor pid=36442) [Client 4] loss:2.341111183166504, Client 4 accuracy:0.10000000149011612 [repeated 3x across cluster]


(ClientAppActor pid=36442) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead. [repeated 2x across cluster]
(ClientAppActor pid=36442)   super().__init__(**kwargs) [repeated 2x across cluster]


Additionally, it is possible to implement a custom strategy from scratch by implementing the necessary methods and extending `flwr.server.strategy.Strategy`. The required methods for a custom strategy are as follows:
* `num_fit_clients`: returns the number of clients to be selected for the next round of training.
* `num_rounds`: returns the number of rounds of training to perform.
* `on_fit`: called when a client has completed training and returned its updated model. This method should update the global model based on the returned model.
* `on_evaluate`: called when a client has completed an evaluation and returned its evaluation result. This method should aggregate the evaluation results.


You can see an schema of the methods and an example in the following [link](https://flower.ai/docs/framework/tutorial-series-build-a-strategy-from-scratch-pytorch.html#Build-a-Strategy-from-scratch)

# Challenges for Federated Learning

While federated learning can solve problems that traditional centralized machine learning struggles with, such as privacy and reduced hardware requirements, it also presents its own challenges. In this section, we will cover some of these challenges, including the non-IID (independent and identically distributed) nature of the data, the heterogeneous nature of devices, and the limited communication bandwidth.

## Non-IID data
The assumption of independence and identical distribution, or i.i.d., is commonly made in machine learning and statistical analysis. This means that each data point is independent of all other data points, and that the distribution of the data is the same across all data points.

Non-i.i.d. data, on the other hand, violates one or both of these assumptions. This can occur for a variety of reasons. For example, data may be collected in a way that introduces dependencies between data points, such as when data is collected over time or in a specific order. Additionally, the distribution of the data may vary across different subgroups or regions, making it non-i.i.d. Non-i.i.d. data is a common challenge in federated learning because the data is distributed across many devices, and each device may have a different distribution of data due to variations in data collection methods or data sources. As a result, traditional machine learning algorithms may not perform well on non-i.i.d. data.

In a non-IID data problem (see Figure 1(a)), "non-IIDness" (see Figure 1(c)) refers to the presence of couplings (such as co-occurrence, neighborhood, dependency, linkage, correlation, and causality) and heterogeneities within and between two or more aspects, such as entities, entity classes, entity properties (variables), processes, facts, and states of affairs, or other types of entities or properties (such as learners and learned results) that appear or are produced prior to, during, and after a target process (such as a learning task). Conversely, IIDness ignores or simplifies these relationships, as shown in Figure 1(b).

![Diagram with IID and non-IID data](https://datasciences.org/wp-content/themes/dslabNew/images/datasciences/IIDness.png)
Credit: [Source of the image](https://datasciences.org/non-iid-learning/)

Non-i.i.d. data can be more challenging to work with than i.i.d. data because standard statistical assumptions and techniques may not be applicable. Therefore, special techniques may need to be employed to analyze non-i.i.d. data, which may include techniques that take into account the dependencies between data points or the varying data distributions.

In this context, non-i.i.d. data refers to the fact that the data on each device may differ in terms of distribution, characteristics, and relevance to the task at hand. For instance, the data on one device may comprise mainly images of dogs, while the data on another device may consist mainly of images of cats. This can pose a challenge in training a model that performs well on all the devices because the data on each device can vary significantly from the data on the other devices.

To address non-i.i.d. data in federated learning, special techniques are often employed to weigh the contributions of each device's data to the overall model, or to adjust the model's parameters in a way that considers the differences in the data. Furthermore, techniques such as data augmentation and transfer learning could help to generalize the model beyond the device's data.

When discussing Flower, the approach to addressing this problem would involve [implementing](https://flower.ai/docs/framework/how-to-implement-strategies.html) a custom strategy, similar to the following example, that uses a custom aggregation of the results.


In [5]:
from flwr.common import EvaluateRes, FitRes, Scalar
from flwr.server.client_proxy import ClientProxy
from flwr.server import ServerConfig

class AggregateCustomMetricStrategy(fl.server.strategy.FedAvg):
    #aggregate_evaluate is responsible for aggregating the results 
    #returned by the clients that were selected and asked to evaluate in configure_evaluate.
    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation accuracy using weighted average."""

        if not results:
            return None, {}

        # Call aggregate_evaluate from base class (FedAvg) to aggregate loss and metrics
        aggregated_loss, aggregated_metrics = super().aggregate_evaluate(server_round, results, failures)

        # Weigh accuracy of each client by number of examples used
        accuracies = [r.metrics["accuracy"] * r.num_examples for _, r in results]
        examples = [r.num_examples for _, r in results]

        # Aggregate and print custom metric
        aggregated_accuracy = sum(accuracies) / sum(examples)
        print(f"Round {server_round} accuracy aggregated from client results: {aggregated_accuracy}")

        # Return aggregated loss and metrics (i.e., aggregated accuracy)
        return aggregated_loss, {"accuracy": aggregated_accuracy}


def server_fn(context: Context):
    # instantiate the model
    model = generate_ann()
    params = get_parameters(model)
    del model
    global_model_init = ndarrays_to_parameters(params)

    # Create strategy and run server
    strategy = AggregateCustomMetricStrategy(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=2,
        min_available_clients=NUM_CLIENTS,
        initial_parameters=global_model_init,
        evaluate_fn=get_test_loader(testloader[0]),
    )
# Construct ServerConfig
    config = ServerConfig(num_rounds=num_rounds)

    # Wrap everything into a `ServerAppComponents` object
    return ServerAppComponents(strategy=strategy, config=config)


# Create your ServerApp
server_app = ServerApp(server_fn=server_fn)

#Start the simulation
fl.simulation.run_simulation(
    server_app=server_app, client_app=client_app, num_supernodes=NUM_CLIENTS
)


/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0954 - loss: 2.4960


INFO :      initial parameters (loss, other metrics): 2.503070116043091, {'accuracy': 0.09000000357627869}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation round 0 with loss 2.503070116043091 / accuracy 0.09000000357627869


(ClientAppActor pid=36568) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(ClientAppActor pid=36568)   super().__init__(**kwargs)


...

INFO :      fit progress: (3, 2.3216006755828857, {'accuracy': 0.11999999731779099}, 12.777037666004617)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation round 3 with loss 2.3216006755828857 / accuracy 0.11999999731779099
(ClientAppActor pid=36568) [Client 3] evaluate, config: {} [repeated 6x across cluster]
(ClientAppActor pid=36567) [Client 0] loss:2.2168092727661133, Client 0 accuracy:0.23999999463558197 [repeated 5x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 14.12s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.364131530125936
INFO :      		round 2: 2.278221050898234
INFO :      		round 3: 2.2986131509145102
INFO :      	History (loss, centralized):
INFO :      		round 0: 2.503070116043091
INFO :      		round 1: 2.3766465187072754
INFO :      		round 2: 2.3183960914611816
INFO :      		round 3: 2.3216006755828857
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.08999999860922496),
INFO :      	              (2, 0.1933333327372869),
INFO :      	              (3, 0.12666666507720947)]}
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.09000000357627869),
INFO :      	              (1, 0.10000000149011612),
INFO :      	              (2, 0.11999999731779099),
INFO :      	              (3, 0.11999999731779099)]}
INFO :

Round 3 accuracy aggregated from client results: 0.12666666507720947
(ClientAppActor pid=36570) [Client 8] fit, config: {} [repeated 2x across cluster]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0745 - loss: 2.3073   [repeated 6x across cluster]
(ClientAppActor pid=36570) [Client 1] evaluate, config: {} [repeated 2x across cluster]
(ClientAppActor pid=36570) [Client 1] loss:2.303380250930786, Client 1 accuracy:0.09000000357627869 [repeated 3x across cluster]


(ClientAppActor pid=36570) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead. [repeated 2x across cluster]
(ClientAppActor pid=36570)   super().__init__(**kwargs) [repeated 2x across cluster]


## Heterogeneity of the devices

The heterogeneity of the devices in the network, which means they may have different hardware and software configurations and may be running different versions of the operating system, is one of the problems of federated learning. This can lead to several problems, including:

* Inefficient communication: Different devices may have varying network speeds and bandwidth, which can make it difficult to transmit model updates between devices in a timely manner.

* Incompatible updates: If different devices are running different versions of the operating systems, they may not be able to exchange model updates due to compatibility issues.

* Data heterogeneity: The data on different devices may differ in terms of quality, quantity, and format, making it challenging to train a model that generalizes well across all devices.



To mitigate the impact of heterogeneous devices in federated learning, researchers are developing techniques such as device-aware aggregation algorithms and communication optimization. These techniques aim to address issues such as inefficient communication and incompatible updates resulting from differences in network speeds, bandwidth, operating system versions, and data heterogeneity across the devices.


Consider a network of five devices (A, B, C, D, and E) that are participating in federated learning to train a global model. Each device has its own data and trains a local model on that data. The local models are then transmitted back to a central server, where they are aggregated and used to update the global model.


In the above scenario, the participating devices (A, B, C, D, and E) in the federated learning network are heterogeneous in nature, meaning they possess different hardware and software configurations. For instance, Device A and Device B may be running distinct versions of the operating system, and Device C may have a slower network connection in comparison to the other devices.


This heterogeneity in the devices can create challenges in the federated learning process. For instance, Device A may face difficulty sending its local model update to the server because of compatibility issues with Device B, and Device C may experience a slower transmission due to its slower network connection.


To overcome the challenges posed by heterogeneous devices in federated learning, researchers are developing techniques to mitigate their impact. These techniques may include device-aware aggregation algorithms, which take into account the different hardware and software configurations of the devices, and communication optimization techniques such as data compression and intelligent routing. By adapting the way that data is aggregated and transmitted, these techniques can help to ensure that all devices are able to contribute effectively to the global model, regardless of their individual characteristics.



It is also worth mentioning that a local configuration can be provided to the `Clients` by means of the `config` parameter of the function in the `FlowerClient`. This parameter is a Python `Dict` which holds values that can be used internally for different purposes, such as limiting the number of epochs on certain clients or establishing the number of rounds.


The modification for the strategy in this case would require the use of parameter `on_fit_config` to indicate the function to retrieve the correct configuration.


```python

...

def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,
    }
    return config

...

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=10,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(model)),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

...
```

However, sometimes limiting the number of rounds or the number of epochs for each client is not enough, especially when the number of clients is too large to handle. In such cases, it may be necessary to reduce the number of clients used for training and evaluation. For instance, consider a scenario where there are 1000 clients, each with only 50 samples for training and 10 for evaluation. Although the amount of data in each client is limited, the communication overhead can still be overwhelming. In such cases, it is better to train for a longer time with a smaller number of clients in each round.

In [6]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config

def server_fn(context: Context):
    # instantiate the model
    model = generate_ann()
    params = get_parameters(model)
    del model

    # Create strategy and run server
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=0.025,  # Train on 25 clients (each round)
        fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
        min_fit_clients=20,
        min_evaluate_clients=40,
        min_available_clients=NUM_CLIENTS,
        initial_parameters=fl.common.ndarrays_to_parameters(params),
        on_fit_config_fn=fit_config
    )
   
     # Construct ServerConfig
    config = ServerConfig(num_rounds=num_rounds)

    # Wrap everything into a `ServerAppComponents` object
    return ServerAppComponents(strategy=strategy, config=config)

# Create your ServerApp
server_app = ServerApp(server_fn=server_fn)

#Start the simulation
fl.simulation.run_simulation(
    server_app=server_app, client_app=client_app, num_supernodes=NUM_CLIENTS
)

/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/utils/file_utils.py:115: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  archive.extractall(
/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 23 clients (

(ClientAppActor pid=36658) [Client 356] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=36659) [Client 46] fit, config: {'local_epochs': 3, 'server_round': 1}


(ClientAppActor pid=36658) 2025-02-16 17:35:29.343181: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
(ClientAppActor pid=36658) 	 [[{{node IteratorGetNext}}]]
(ClientAppActor pid=36658) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
(ClientAppActor pid=36658)   self.gen.throw(value)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2222 - loss: 2.4302  
(ClientAppActor pid=36660) [Client 808] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 4x across cluster]
(ClientAppActor pid=36658) [Client 581] fit, config: {'local_epochs': 3, 'server_round': 1} [repeated 7x across cluster]


(ClientAppActor pid=36659) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead. [repeated 3x across cluster]
(ClientAppActor pid=36659)   super().__init__(**kwargs) [repeated 3x across cluster]
(ClientAppActor pid=36658) 2025-02-16 17:35:34.770937: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence [repeated 8x across cluster]
(ClientAppActor pid=36658) 	 [[{{node IteratorGetNext}}]] [repeated 8x across cluster]
(ClientAppActor pid=36659) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You ma

...

INFO :      aggregate_fit: received 1 results and 24 failures
INFO :      configure_evaluate: strategy sampled 50 clients (out of 1000)
ERROR :     An exception was raised when processing a message by RayBackend


(ClientAppActor pid=36657) [Client 25] loss:2.5321390628814697, Client 25 accuracy:0.0 [repeated 7x across cluster]


ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     cannot access local variable 'future' where it is not associated with a value
ERROR :     cannot access local variable 'future' where it is not associated with a value
ERROR :     cannot access local variable 'future' where it is not associated with a value
ERROR :     Traceback (most recent call last):
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/server/superlink/fleet/vce/backend/raybackend.py", line 166, in process_message
    future = self.pool.submit(
             ^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 458, in submit
    actor = self.pool.pop()
            ^^^^^^^^^^^^^^^
IndexError: pop from empty list

During handling of the above exception, another exception occurred:

Traceback

(ClientAppActor pid=36657) [Client 76] evaluate, config: {} [repeated 2x across cluster]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.0000e+00 - loss: 2.9001 [repeated 3x across cluster]
(ClientAppActor pid=36657) [Client 76] loss:2.9001033306121826, Client 76 accuracy:0.0 [repeated 2x across cluster]


(ClientAppActor pid=36657) 2025-02-16 17:35:45.982390: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
(ClientAppActor pid=36657) 	 [[{{node IteratorGetNext}}]]


In addition to the techniques mentioned earlier, federated transfer learning, secure aggregation, and data augmentation are other approaches that can help in the scaling of the federated learning system. The limitation of resources, including bandwidth, storage, and computation power, is one of the main challenges of federated learning.


### Exercise

As evident from the previous results, the outcomes are not remarkable, mainly attributed to the limited number of patterns for each client. In response, suggest an alternative architecture for the network and experiment with at least four different configurations for the fraction_fit and evaluate. Subsequently, analyze the data and draw conclusions from your findings.

In [ ]:
# Define a new architecture for the neural network
def generate_new_ann():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    return model

# def get_parameters(model):
#     return model.get_weights()

# def set_parameters(model, parameters):
#     model.set_weights(parameters)

def get_test_loader(test_data):
    def evaluate_fn(server_round, parameters, config):
        model = generate_new_ann()
        set_parameters(model, parameters)
        loss, accuracy = model.evaluate(test_data[0], test_data[1])
        return loss, {"accuracy": accuracy}
    return evaluate_fn

# Experiment with different configurations for fraction_fit and fraction_evaluate
configurations = [
    {"fraction_fit": 0.1, "fraction_evaluate": 0.1},
    {"fraction_fit": 0.2, "fraction_evaluate": 0.2},
    {"fraction_fit": 0.3, "fraction_evaluate": 0.3},
    {"fraction_fit": 0.4, "fraction_evaluate": 0.4},
]

results = []

# Define the number of clients
NUM_CLIENTS = 5
num_rounds = 3

for config in configurations:
    def server_fn(context: Context):
        # Generate the model and parameters
        model = generate_new_ann()
        params = get_parameters(model)
        del model
        global_model_init = ndarrays_to_parameters(params)

        # Create FedAvg strategy
        strategy = FedAvg(
            fraction_fit=config["fraction_fit"],
            fraction_evaluate=config["fraction_evaluate"],
            min_fit_clients=3,
            min_evaluate_clients=2,
            min_available_clients=NUM_CLIENTS,
            initial_parameters=global_model_init,
            evaluate_fn=get_test_loader(testloader[0]),
        )

        # Define ServerConfig
        server_config = fl.server.ServerConfig(num_rounds=num_rounds)

        # Return the configuration and strategy for this server
        return ServerAppComponents(strategy=strategy, config=server_config)

    # Create Server
    server_app = ServerApp(server_fn=server_fn)

    # Create Client
    client_app = ClientApp(client_fn=client_fn)

    # Start the simulation
    history = fl.simulation.run_simulation(
        server_app=server_app, client_app=client_app, num_supernodes=NUM_CLIENTS
    )

    results.append({
        "config": config,
        "history": history
    })

# Analyze the results
for result in results:
    config = result["config"]
    history = result["history"]
    print(f"Configuration: {config}")
    print(f"History: {history}")

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.0000e+00 - loss: 2.4493


INFO :      initial parameters (loss, other metrics): 2.4492874145507812, {'accuracy': 0.0}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 5)
(raylet) [2025-02-16 17:35:55,258 E 36789 10724901] file_system_monitor.cc:116: /tmp/ray/session_2025-02-16_17-35-49_841862_36200 is over 95% full, available space: 22.882 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.
(ClientAppActor pid=36808) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(ClientAppActor pid=36808)   super().__init__(**kwargs)
ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     An exception was r

(ClientAppActor pid=36809) [Client 3] fit, config: {}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.0000e+00 - loss: 2.4493


INFO :      fit progress: (1, 2.4492874145507812, {'accuracy': 0.0}, 7.932210583996493)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 5)
ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     ray::ClientAppActor.run() (pid=36809, ip=127.0.0.1, actor_id=fc38396f135da7914bec5dd201000000, repr=<flwr.simulation.ray_transport.ray_actor.ClientAppActor object at 0x106226ff0>)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/client/client_app.py", line 143, in __call__
    return self._call(message, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/client/client_app.py", line 126, in ffn
    out_message = handle_legacy_message_from_msgtype(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/cl

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.0000e+00 - loss: 2.4083


INFO :      fit progress: (3, 2.4083425998687744, {'accuracy': 0.0}, 9.814548000053037)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 5)
ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     ray::ClientAppActor.run() (pid=37043, ip=127.0.0.1, actor_id=ceac222469808d4e2ba1be5001000000, repr=<flwr.simulation.ray_transport.ray_actor.ClientAppActor object at 0x10591ee10>)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/client/client_app.py", line 143, in __call__
    return self._call(message, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/client/client_app.py", line 126, in ffn
    out_message = handle_legacy_message_from_msgtype(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/flwr/cl

(ClientAppActor pid=37044) [Client 3] fit, config: {} [repeated 8x across cluster]
(ClientAppActor pid=37044) [Client 4] evaluate, config: {} [repeated 5x across cluster]


(ClientAppActor pid=37043) /Users/albertolandi/anaconda3/envs/ml2/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead. [repeated 2x across cluster]
(ClientAppActor pid=37043)   super().__init__(**kwargs) [repeated 2x across cluster]


Configuration: {'fraction_fit': 0.1, 'fraction_evaluate': 0.1}
History: None
Configuration: {'fraction_fit': 0.2, 'fraction_evaluate': 0.2}
History: None
Configuration: {'fraction_fit': 0.3, 'fraction_evaluate': 0.3}
History: None
Configuration: {'fraction_fit': 0.4, 'fraction_evaluate': 0.4}
History: None
